In [1]:
import time

import cv2
import pandas as pd
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [ ]:
def preprocess_image(img):
    # Apply median filter for noise removal
    img_median = cv2.medianBlur(img, 3)

    # Convert the image to grayscale for OTSU thresholding
    img_gray = cv2.cvtColor(img_median, cv2.COLOR_BGR2GRAY)

    # Apply OTSU thresholding for binarization
    _, binary_image = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Convert binary image to 3 channels
    binary_image_3_channels = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)

    # Display the original, median-filtered, and binarized images for comparison
    # plt.figure(figsize=(10, 5))
    # plt.subplot(131), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
    # plt.subplot(132), plt.imshow(cv2.cvtColor(img_median, cv2.COLOR_BGR2RGB)), plt.title('Median Filtered')
    # plt.subplot(133), plt.imshow(binary_image_3_channels), plt.title('Binarized (3 channels)')
    # plt.show()

    return binary_image_3_channels

# Define transforms to convert images to PyTorch tensors and normalize them
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # These values are for ImageNet normalization
])

In [ ]:
def read_images_from_excel(excel_file_path):
    # Read Excel file
    data = pd.read_excel(excel_file_path)

    images = []
    labels = []

    i = 0
    print("Reading images.")
    # Iterate over each row in the Excel file
    for index, row in data.iterrows():
        filename = row['File Name']
        gender = row['Gender']

        # Read image using cv2
        image = cv2.imread('./images/' + filename)

        # Preprocess image
        if image is not None:
            image = preprocess_image(image)
            image = transform(image).unsqueeze(0)
            images.append(image)
            labels.append(gender)
            print(i+1, '/3840')
            i += 1

    return images, labels

In [ ]:
start = time.time()
excel_file_path = 'updated_excel_file2.xlsx'
images, labels = read_images_from_excel(excel_file_path)
end = time.time()
print(f"Done reading and processing {len(images)} images in {end-start}s.")

Reading images.
1 /3840
2 /3840
3 /3840
4 /3840
5 /3840
6 /3840
7 /3840
8 /3840
9 /3840
10 /3840
11 /3840
12 /3840
13 /3840
14 /3840
15 /3840
16 /3840
17 /3840
18 /3840
19 /3840
20 /3840
21 /3840
22 /3840
23 /3840
24 /3840
25 /3840
26 /3840
27 /3840
28 /3840
29 /3840
30 /3840
31 /3840
32 /3840
33 /3840
34 /3840
35 /3840
36 /3840
37 /3840
38 /3840
39 /3840
40 /3840
41 /3840
42 /3840
43 /3840
44 /3840
45 /3840
46 /3840
47 /3840
48 /3840
49 /3840
50 /3840
51 /3840
52 /3840
53 /3840
54 /3840
55 /3840
56 /3840
57 /3840
58 /3840
59 /3840
60 /3840
61 /3840
62 /3840
63 /3840
64 /3840
65 /3840
66 /3840
67 /3840
68 /3840
69 /3840
70 /3840
71 /3840
72 /3840
73 /3840
74 /3840
75 /3840
76 /3840
77 /3840
78 /3840
79 /3840
80 /3840
81 /3840
82 /3840
83 /3840
84 /3840
85 /3840
86 /3840
87 /3840
88 /3840
89 /3840
90 /3840
91 /3840
92 /3840
93 /3840
94 /3840
95 /3840
96 /3840
97 /3840
98 /3840
99 /3840
100 /3840
101 /3840
102 /3840
103 /3840
104 /3840
105 /3840
106 /3840
107 /3840
108 /3840
109 /3840
11

In [ ]:
len(images)

3840

In [ ]:
# Check if GPU is available
device = torch.device("cuda")

# Define a custom dataset class to load images
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Create a custom dataset instance
dataset = CustomDataset(images, labels)

# Create a data loader
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained MobileNet model
mobilenet = models.mobilenet_v2(pretrained=True)

# Move the model to the GPU
mobilenet.to(device)

# Remove the classifier layer (fully connected layer) from the MobileNet model
mobilenet_features = torch.nn.Sequential(*list(mobilenet.children())[:-1])

# Put the model in evaluation mode
mobilenet_features.eval()

# Extract features from images using MobileNet
features_list = []
labels_list = []

with torch.no_grad():
    for images_batch, labels_batch in data_loader:
        # Move images to the GPU
        images_batch = images_batch.to(device)
        features_batch = mobilenet_features(images_batch)
        features_list.append(features_batch.cpu())  # Move features back to CPU
        labels_list.append(labels_batch)

# Concatenate features and labels
features = torch.cat(features_list, dim=0)
labels = torch.cat(labels_list, dim=0)

# Convert features and labels to numpy arrays
features_array = features.numpy()
labels_array = labels.numpy()